<a href="https://colab.research.google.com/github/LiamAMcNamara/HMMs-Lab3/blob/Part2/McNamara_Lab_3_HMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CSE 5522 Lab #3: Hidden Markov Models**

Liam McNamara.308

June 11, 2020

---
---

**Part 1 (50 Points):** Implement the Viterbi Algorithm for HMMs for Eisner's Ice Cream Problem (predict whether each day is hot or cold based on the number of ice creams eaten). See elsner.hmm.xls for a bit more discussion on the problem. Remeber that the Viterbi algorithm computes the most likely sequence for an input.

Your solution should be able to handle variable length sequences (in the range of 3-5).

There are 3 files provided for you that have observation/emission probabilities, transition probabilities, and test sequence data (for evaluation), respectively.

Please read the probabilities and observations from their Carmen URLs, *do not hard-code the values*. Refer back to lab 2 for exampes of working with CSV files and how to download them from URLs.

The observation and transition probabilities have rows being the variable of interest, and columns being the conditioning variables. For example, P(2|H) is in the 3rd row (including header), 3rd column (including row label). The columns sum up to 1.

The test data has one line per sequence. When a sequence is less than five observations long, the last  columns are filled with zeros.



Set up the environment

In [222]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random as rdm

Read in the provided data from Carmen URL

In [195]:

emissionURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/observationProbsLecture.csv'
emission_dataframe = pd.read_csv(emissionURL)
transitionURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/transitionProbsLecture.csv'
transition_dataframe = pd.read_csv(transitionURL)
testDataURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/testDataLecture.csv'
testData_dataframe = pd.read_csv(testDataURL)

In [196]:
emissionURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/observationProbs.csv'
emission_dataframe = pd.read_csv(emissionURL)
transitionURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/transitionProbs.csv'
transition_dataframe = pd.read_csv(transitionURL)
testDataURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/testData.csv'
testData_dataframe = pd.read_csv(testDataURL)

Print out the top of the dataframes to make sure the data loaded correctly. The data tables should have the following format:

Emission Probabilities [3, 3]: 3 columns (P(x|...), C, H), and 3 rows corresponding the the number of ice creams

Transition Probabilities [3, 4]: 4 columns (P(x|...), C, H, START), and 3 rows (C, H, STOP)

Test Sequence Data [10, 6]: 6 columns (SeqNumber, Obs1...5), and 10 rows corresponing the the sequence sample number.

Note: The test data has one line per sequence. When a sequence is less than five observations long, the last columns are filled with zeros.

In [197]:
print("Emission Probabilities")
display(emission_dataframe.shape)
display(emission_dataframe.head())
print('\n')
print("Transition Probabilities")
display(transition_dataframe.shape)
display(transition_dataframe.head())
print('\n')
print("Test Sequence Data")
display(testData_dataframe.shape)
display(testData_dataframe.head())

Emission Probabilities


(3, 3)

,P(x|...),C,H
0,1,0.6407,0.0002
1,2,0.1481,0.5341
2,3,0.2122,0.4657




Transition Probabilities


(3, 4)

,P(x|...),C,H,START
0,C,0.87,0.47,0.5
1,H,0.07,0.47,0.5
2,STOP,0.06,0.06,0.0




Test Sequence Data


(10, 6)

,SeqNumber,Obs1,Obs2,Obs3,Obs4,Obs5
0,1,2,3,3,2,3
1,2,2,3,2,2,0
2,3,3,1,3,3,1
3,4,2,1,1,0,0
4,5,1,1,1,2,3


Initialize matrix variables and shorthand for emission and transition arrays

In [198]:
# In our class notes we referred to the transition matrix as 'A'
# and the emission/observation matrix as 'B'. So I will be doing the same.
A = transition_dataframe.to_numpy()
A = np.delete(A, 0, 1)  # remove extra (label) column included in the dataframe
                        # this column included (C, H, STOP)
B = np.array(emission_dataframe)
B = np.delete(B, 0, 1)  # remove extra (label) column included in the dataframe
                        # this column included (1, 2 , 3)

sequences = testData_dataframe.to_numpy()
sequences = np.delete(sequences, 0, 1)  # remove extra (label) column included in
                                        # the df. This column inc. (1 ... n seqs)

print(A)
print(B)

[[0.87 0.47 0.5]
 [0.07 0.47 0.5]
 [0.06 0.06 0.0]]
[[6.407e-01 2.000e-04]
 [1.481e-01 5.341e-01]
 [2.122e-01 4.657e-01]]


Find most likely sequence of hidden varialbles given the obervation sequence.

Below you will see outputs of the M and C-Matrix. For the C-Matrix, I incremented the values for printing so that they match the notation we used in lecture. However in my implementation, what prints as [1, 2, 1] is actually [0,  1, 0]. This is so that I can use the values in the C-matrix as indexes in the backward-phase. The same goes for the most likely sequence.

In [199]:
for n in range(len(sequences)): #len(sequences)
  seq = sequences[n]
  seq = seq[seq != 0]
  print(f'Observed sequence {n+1}: {seq}\n')
  # M.shape = [sequence length, # of hidden variables]
  m_matrix = np.zeros((len(seq), emission_dataframe.shape[1]-1)) 
  # In this case, Sequence of length 3-5 with 2 possible hidden vairable values (H, C)

  # C.shape = [sequence length - 1, # hidden variables]
  c_matrix = np.delete(m_matrix, 0, 0)
  # In this case, Sequence of length 5-1 with 2 hidden variable values
  c_matrix = c_matrix.transpose()
  # print(m_matrix)
  # print(c_matrix)

  seq = seq - 1 #decrement seq for proper indexing
  forward(m_matrix, c_matrix)


Observed sequence 1: [2 3 3 2 3]

Forward phase calculations:
M[0][0]: 0.5 * 0.1481
M[0][1]: 0.5 * 0.5341
M[2][1]: 0.2122 * 0.18886983289357961
M[2][2]: 0.4657 * 0.36796687188507765
M[3][1]: 0.2122 * 0.1649071012306746
M[3][2]: 0.4657 * 0.3809122556569919
M[4][1]: 0.1481 * 0.14334479886912377
M[4][2]: 0.5341 * 0.39256085578334693
M[5][1]: 0.2122 * 0.07999072256804153
M[5][2]: 0.4657 * 0.4267866211414028
M-Matrix:
[[0.21709176 0.78290824]
 [0.18954839 0.81045161]
 [0.16476414 0.83523586]
 [0.09194336 0.90805664]
 [0.07868236 0.92131764]]
C-Matrix (transposed):
[[1. 2.]
 [1. 2.]
 [1. 2.]
 [1. 2.]]
Most Likely Sequence: [2 2 2 2 2]
___________________________________________________________

Observed sequence 2: [2 3 2 2]

Forward phase calculations:
M[0][0]: 0.5 * 0.1481
M[0][1]: 0.5 * 0.5341
M[2][1]: 0.2122 * 0.18886983289357961
M[2][2]: 0.4657 * 0.36796687188507765
M[3][1]: 0.1481 * 0.1649071012306746
M[3][2]: 0.5341 * 0.3809122556569919
M[4][1]: 0.1481 * 0.0932460344410975
M[4][2]: 0.

Forward Phase: Calculate values for the M-Matrix. One step at a time. Starting at first state and moving forward. Filling in the C-Marix as we go.

Choose value for last state by maximizing value of hidden state

In [200]:
def forward(m_matrix, c_matrix):
  # use priors to set first row of M matrix
  print("Forward phase calculations:")
  for i in range(B.shape[1]):
    m_matrix[0,i] = A[i,A.shape[1]-1] * B[seq[0], i]
    print(f'M[0][{i}]: {A[i,A.shape[1]-1]} * {B[seq[0], i]}')

  normalizeRow(m_matrix, 0);
  vect = np.zeros(3)
  for s in range(1, len(seq)):
    for k in range(B.shape[1]):
      for j in range(B.shape[1]):
        vect[j] = (A[j][k] * m_matrix[s-1][j])
      max = np.max(vect)
      print(f'M[{s+1}][{k+1}]: {B[seq[s], k]} * {max}')
      m_matrix[s][k] = B[seq[s], k] * max
      c = np.where(vect == max)
      c_matrix[k][s-1]=c[0][0]
    normalizeRow(m_matrix, s)

  c_matrix_print = c_matrix + 1
  print(f'M-Matrix:\n{m_matrix}')
  print(f'C-Matrix (transposed):\n{c_matrix_print.transpose()}')
  # convert C-Matrix to int so we can use its values as indexes
  c_matrix = c_matrix.astype(int)
  backward(m_matrix, c_matrix)

**Backward Phase**: Use final row of M-matrix to find final member of the most likely sequence. From this, us C-Matrix to work backwards and find what the values of the preceding hidden states should be.

In [201]:
def backward(m_matrix, c_matrix):
  MLS = np.zeros_like(seq)
  last_row = m_matrix[len(seq)-1]
  m_max = np.max(last_row)
  last_hidden = np.where(last_row == m_max)
  MLS[len(seq)-1] = last_hidden[0][0]
  #c_matrix = c_matrix.astype(int)
  #print(MLS)
  for i in range(len(seq) - 2, -1, -1):
    #print("i:", i)
    #print(MLS[i+1])
    x = c_matrix[MLS[i+1]][i]
    MLS[i] = x
  MLS_print = MLS + 1
  print(f'Most Likely Sequence: {MLS_print}')
  print('___________________________________________________________\n')

In [202]:
def normalizeRow(m, i):
  # Normalize each row in M-Matrix (each obseervation) to prevent underflow
  norm_m = m  #python uses pass by reference so this will change m
  row = m[i]
  row = row / np.sum(row)
  norm_m[i] = row


**Part 2 (50 points)**: Using the same network, implement a likelihood samples for approximate inference. For any test sequence, sample complete sequences of the hidden states *n* times, where *n* can range from 10 to 100,000 samples. The goal is to approximate the likelihood of *all possible* sequences.

Assuming the Viterbi sequence is "correct", how long (how many samples) does it take the sampler to converge so that the (estimated) most likely sample you get matches the Viterbi sequence?

---
---


Diregard STOP probabilities and normalize remaining transition probabilities

In [217]:
#print(f'A:\n{A}')
#remove STOP probabilities (bottom row)
transitionProbs = np.delete(A, A.shape[0]-1, 0)
#remove and store START priors (right-most column)
priors = transitionProbs[:,A.shape[1]-1]
transitionProbs = np.delete(transitionProbs, transitionProbs.shape[1]-1, 1)
#normalize across rows
normalizeRow(transitionProbs, 0)
normalizeRow(transitionProbs, 1)
print(f'transitionProbs:\n{transitionProbs}')
#print(f'priors:\n{priors}')

transitionProbs:
[[0.6492537313432837 0.35074626865671643]
 [0.12962962962962962 0.8703703703703702]]


Sample

In [272]:
for n in range(len(sequences)): #len(sequences)
  seqt = sequences[n]
  seqt = seqt[seqt != 0]
  print(f'Observed sequence {n+1}: {seqt}')
  seqt = seqt - 1 # for indexing

  sampseq = np.zeros_like(seqt)
  mlsw = np.zeros_like(seqt)
  bestweight = 0
  for i in range(10):
    weight = weightedInference(seqt, sampseq)
    if weight > bestweight:
      mlsw = np.copy(sampseq)
      bestweight = weight
    sampseq_print = sampseq + 1
    #print(f'Sampled sequence {i}: {sampseq_print}')
    #print(f'weight {i}: {weight}\n')
  print()
  mlsw_print = mlsw + 1
  print(f'mlsw: {mlsw_print}')
  print(f'bestweight: {bestweight}\n')
  print("_________________________________")
  

Observed sequence 1: [2 3 3 2 3]

mlsw: [2 2 2 2 2]
bestweight: 0.028811367344428896

_________________________________
Observed sequence 2: [2 3 2 2]

mlsw: [2 2 2 2]
bestweight: 0.07095352427853971

_________________________________
Observed sequence 3: [3 1 3 3 1]

mlsw: [1 1 1 2 1]
bestweight: 0.008608082984016935

_________________________________
Observed sequence 4: [2 1 1]

mlsw: [1 1 1]
bestweight: 0.06079453016900001

_________________________________
Observed sequence 5: [1 1 1 2 3]

mlsw: [1 1 1 2 2]
bestweight: 0.06541735611918584

_________________________________
Observed sequence 6: [1 3 1 1]

mlsw: [1 1 1 1]
bestweight: 0.05580968246254461

_________________________________
Observed sequence 7: [3 2 3]

mlsw: [2 2 2]
bestweight: 0.115833733309

_________________________________
Observed sequence 8: [2 2 1 1]

mlsw: [1 1 1 1]
bestweight: 0.009003669918028903

_________________________________
Observed sequence 9: [1 3 2 3 3]

mlsw: [1 2 2 2 1]
bestweight: 0.015748353595

In [269]:
def weightedInference(seqt, sampseq): # NOTE: (for calulation) 0 = C, 1 = H
  
  # sample first hidden state using priors
  x = rdm.random()
  if x < 0.5: sampseq[0] = 0
  else: sampseq[0] = 1

  #print(f'sampled day1: {sampseq[0]}')
  weight = B[seqt[0]][sampseq[0]]
  #print(f'weight: {weight}')
  # sample remaining states using transition probabilities
  for i in range(1, len(seqt)):
    x = rdm.random()
    if x < A[sampseq[i-1], 0]: sampseq[i] = 0
    else: sampseq[i] = 1
      
    #print(f'sampled day{i+1}: {sampseq[i]}')
    #print(f'THIS weight: {B[seqt[i]][sampseq[i]]}')
    weight *= B[seqt[i]][sampseq[i]]
    #print(f'weight: {weight}')
  
  return weight